### Model Training

In [1]:
import csv
import pandas as pd

In [2]:
try:
    import models
except ImportError:
    print('Please upload models.py to you current directory')
    raise

2024-04-21 19:22:51.720362: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-21 19:22:51.769732: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 19:22:52.531699: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
df = pd.read_pickle("./data/cve")

In [4]:
train = df[:120000]
test = df[120000:]

In [10]:
from models import BERTmodel

In [11]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A100 80GB PCIe


In [14]:
integrity_model = BERTmodel('integrity', train, test)

In [16]:
integrity_model.train(epochs=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i


======== Epoch 1 / 2 ========
Training...
  Batch 2,000  of  13,500.    Elapsed: 0:12:21.
  Batch 4,000  of  13,500.    Elapsed: 0:25:46.
  Batch 6,000  of  13,500.    Elapsed: 0:39:10.
  Batch 8,000  of  13,500.    Elapsed: 0:52:34.
  Batch 10,000  of  13,500.    Elapsed: 1:05:59.
  Batch 12,000  of  13,500.    Elapsed: 1:19:26.

  Average training loss: 0.37
  Training epoch took: 1:29:31

Running Validation...
  Accuracy: 0.89
  Validation Loss: 0.37
  Validation took: 0:03:16

======== Epoch 2 / 2 ========
Training...
  Batch 2,000  of  13,500.    Elapsed: 0:13:22.
  Batch 4,000  of  13,500.    Elapsed: 0:39:20.
  Batch 6,000  of  13,500.    Elapsed: 1:12:43.
  Batch 8,000  of  13,500.    Elapsed: 1:46:07.
  Batch 10,000  of  13,500.    Elapsed: 2:19:32.
  Batch 12,000  of  13,500.    Elapsed: 2:52:59.

  Average training loss: 0.28
  Training epoch took: 3:18:03

Running Validation...
  Accuracy: 0.90
  Validation Loss: 0.36
  Validation took: 0:08:32

Training complete!
Total tr

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.37,0.37,0.89,1:29:31,0:03:16
2,0.28,0.36,0.90,3:18:03,0:08:32


In [17]:
integrity_model.test()

0.8524788777299538

In [18]:
print(integrity_model.getClassificationStats())

{'accuracy': 0.8524788777299538, 'precision': 0.8524788777299538, 'recall': 0.8524788777299538, 'f1': 0.8524788777299538, 'mcc': 0.7723117340386622, 'cm': array([[ 7325,   158,  1305],
       [  235,  6439,  1850],
       [  666,   413, 12974]])}


In [19]:
integrity_model.saveModel()
integrity_model.savePredictions()

### Push Model to huggingface hub

In [20]:
from huggingface_hub import notebook_login

In [21]:
notebook_login()

In [22]:
from transformers import BertForSequenceClassification

In [23]:
integrity_model = BertForSequenceClassification.from_pretrained("./integrity_model/")

In [24]:
integrity_model.push_to_hub("Vijaybhk/Integrity-BERT")

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Vijaybhk/Integrity-BERT/commit/858e6aeeddb3f2f5897b955881b3a591f6354cee', commit_message='Upload BertForSequenceClassification', commit_description='', oid='858e6aeeddb3f2f5897b955881b3a591f6354cee', pr_url=None, pr_revision=None, pr_num=None)